# ZPropagator - Checking consistency

### Verify that the outputs of the propagation model and the Pennylane counterpart are the same

---

In [1]:
import pennylane as qml 
import jax.numpy as jnp
import numpy as np
from jax import grad

from zprop import propagator, circuits, loss

In [2]:
n_qubit = 4           # Number of qubits
n_iteration = 2       # Depth of the circuit
ansatz = circuits.id9 # Ansatz

In [3]:
# Define Pennylane circuit as normal

def circuit(p_p, p_a):
    ansatz(p_a = p_a, p_p = p_p, n_qubit = n_qubit, n_iteration = n_iteration)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubit)]

q_circuit = qml.QNode(circuit, qml.device('default.qubit', n_qubit))

print(qml.draw(q_circuit)(np.arange(10000), np.arange(10000)))

0: ──H──RY(0.00)──H─╭●──RX(0.00)──────────────────────||──H─╭●──RX(4.00)──────────────────────||─┤
1: ──H──RY(1.00)──H─╰Z─╭●─────────RX(1.00)────────────||──H─╰Z─╭●─────────RX(5.00)────────────||─┤
2: ──H──RY(2.00)──H────╰Z────────╭●─────────RX(2.00)──||──H────╰Z────────╭●─────────RX(6.00)──||─┤
3: ──H──RY(3.00)──H──────────────╰Z─────────RX(3.00)──||──H──────────────╰Z─────────RX(7.00)──||─┤

   <Z>
   <Z>
   <Z>
   <Z>


In [4]:
# Definition of the propagator model

prop_model = propagator.Propagator(n_qubit=n_qubit, ansatz=ansatz, n_iteration=n_iteration)

print(prop_model)

Propagator
  Number of qubits : 4
  Trainable parameters : 8
  Cutoff 1: None | Cutoff 2: None
0: ──H──RY──H─╭●──RX──────────||──H─╭●──RX──────────||─┤  <Z>
1: ──H──RY──H─╰Z─╭●───RX──────||──H─╰Z─╭●───RX──────||─┤  <Z>
2: ──H──RY──H────╰Z──╭●───RX──||──H────╰Z──╭●───RX──||─┤  <Z>
3: ──H──RY──H────────╰Z───RX──||──H────────╰Z───RX──||─┤  <Z>


In [5]:
p_p = np.random.rand(prop_model.n_p_p) # Array of trainable parameters
p_a = np.random.rand(prop_model.n_p_a) # Array of encoded input

#### Circuit output

In [6]:
# Output of the Pennylane model

penny_output = np.array(
q_circuit(
        p_p = p_p,
        p_a = p_a
    )
)

penny_output

array([-0.55490521, -0.04877041, -0.28743651, -0.39587053])

In [7]:
# Lambdify the circuit

prop_func, _ = prop_model.Hlambdify()

# Output of the Propagator model

prop_output  = np.array(prop_func(p_p=p_p, p_a=p_a))

prop_output

array([-0.55490524, -0.04877037, -0.28743654, -0.3958705 ], dtype=float32)

In [8]:
# Are each individual Z-expectation values the same (more or less)
np.isclose(prop_output, penny_output)

array([ True,  True,  True,  True])

---

#### Circuit gradient
Considering the circuit as a function $\mathcal{F}$, the circuit gradient is $\partial{\mathcal{F}}/\partial{p_i}$, namely the amount of change in the output (exp. values) when changing the trainable parameters

In [9]:
def penny_Hprime(p_p, p_a):
    def getZ(p_p, p_a, wire):
        return q_circuit(p_p=p_p, p_a=p_a)[wire]
    
    return [grad(getZ)(p_p, p_a, wire) for wire in prop_model.p_measured_qubit]

np.round(penny_Hprime(p_p, p_a), 3)


array([[-0.006, -0.   , -0.   , -0.   , -0.362, -0.   ,  0.   ,  0.   ],
       [-0.   ,  0.212, -0.   , -0.   , -0.   ,  0.373, -0.   , -0.   ],
       [ 0.   , -0.   ,  0.095,  0.   , -0.   ,  0.   ,  0.321,  0.   ],
       [-0.   ,  0.   , -0.   , -0.184,  0.   ,  0.   ,  0.   ,  0.012]],
      dtype=float32)

#### Loss gradient
Having the circuit as a function, one can quickly compute the gradient of any loss function using the chain rule:

Consider $\mathcal{L}$ a loss function

Its gradient is:

$\displaystyle \partial\mathcal{L}/\partial p_i = \frac{\partial \mathcal{L}}{\partial \mathcal{F}}\frac{\partial \mathcal{F}}{\partial p_i}$

In [10]:
# We test this with the MMD loss function

In [11]:
# MMD hyper-parameters
p_sigma = [1e-3, 1e-2, 1e-1]
noise   = 1e-5

# Target value
Y = 2*np.random.rand(n_qubit) - 1

In [12]:
def mmd_circuit(p_p, p_a):
    output = jnp.array(q_circuit(p_p, p_a))
    return loss.mmd(output, Y, sigmas=p_sigma, noise=noise)

penny_grad = grad(mmd_circuit)(p_p, p_a)

penny_grad

Array([-0.0036348 , -0.1979993 , -0.02794293, -0.03558235, -0.2093063 ,
       -0.34821817, -0.09410448,  0.00236249], dtype=float32)

In [14]:
dLdF = grad(loss.mmd)
dFdp = prop_model.Hprimelambdify()[0]

def grad_prop(p_p, p_a):
    output = np.array(prop_func(p_p, p_a))
    A = np.array(dLdF(output, Y, sigmas=p_sigma, noise=noise))
    B = np.array(dFdp(p_p, p_a))
    return np.einsum('b,ab->a', A, B)

prop_grad = grad_prop(p_p, p_a)

prop_grad

array([-0.0036348 , -0.1979993 , -0.02794298, -0.03558243, -0.2093064 ,
       -0.34821822, -0.09410453,  0.00236249])

In [15]:
# Are the gradient similar?
np.isclose(prop_grad, penny_grad, atol=1e-4)

array([ True,  True,  True,  True,  True,  True,  True,  True])